# 新架構示範 V2 / New Architecture Demo V2

展示 KEEN 新架構的使用方式，包含 IDE 友好的介面和直覺的資料存取  
Demonstrates the usage of KEEN's new architecture with IDE-friendly interface and intuitive data access

---

## 🚀 主要改進 / Key Improvements

✅ **IDE 友好的型別提示**：`session['file'].data.attribute`  
✅ **完整的狀態管理**：所有分析結果自動持久化  
✅ **統一的資料格式**：標準化的 `ParseResult` 和資料模型  
✅ **清晰的職責分離**：型別管理器模式  
✅ **正確的 Scale 處理**：從 TXT 檔案獲取正確的數值 scale

## 1️⃣ 環境設定 / Environment Setup

In [1]:
import sys
from pathlib import Path

# 添加後端路徑到 Python 路徑 / Add backend path to Python path
backend_path = Path.cwd().parent if Path.cwd().name == 'test' else Path.cwd()
sys.path.insert(0, str(backend_path))

print(f"💻 當前工作目錄: {Path.cwd()}")
print(f"💻 後端路徑: {backend_path}")

💻 當前工作目錄: /Users/yangziliang/Git-Projects/keen/backend/test
💻 後端路徑: /Users/yangziliang/Git-Projects/keen/backend


In [2]:
# 匯入新架構的核心模組
from core.experiment_session import ExperimentSession
from core.data_models import TopoData, CitsData, TxtData

print("✅ 新架構模組匯入成功！")
print("✅ New architecture modules imported successfully!")

✅ 新架構模組匯入成功！
✅ New architecture modules imported successfully!


## 2️⃣ 檢查測試檔案 / Check Test Files

In [3]:
# 檢查測試檔案是否存在
testfile_dir = backend_path.parent / "testfile"
txt_files = list(testfile_dir.glob("*.txt"))

print(f"📁 測試檔案目錄: {testfile_dir}")
print(f"📁 找到 {len(txt_files)} 個 TXT 檔案:")

for i, txt_file in enumerate(txt_files):
    print(f"   {i+1}. {txt_file.name}")

if txt_files:
    selected_txt = txt_files[0]
    print(f"\n🎯 將使用: {selected_txt.name}")
else:
    print("❌ 未找到測試檔案！請確保 testfile 目錄中有 .txt 檔案")

📁 測試檔案目錄: /Users/yangziliang/Git-Projects/keen/testfile
📁 找到 1 個 TXT 檔案:
   1. 20250521_Janus Stacking SiO2_13K_113.txt

🎯 將使用: 20250521_Janus Stacking SiO2_13K_113.txt


## 3️⃣ 初始化實驗會話 / Initialize Experiment Session

新架構的核心是 `ExperimentSession`，它整合了所有的型別管理器和提供統一的介面。

In [4]:
# 初始化實驗會話 - 這是新架構的主要入口
if txt_files:
    print("🚀 初始化實驗會話...")
    session = ExperimentSession(str(selected_txt))
    
    print(f"✅ 會話建立成功！")
    print(f"📊 實驗名稱: {session.experiment_name}")
    print(f"📊 TXT 檔案: {session.txt_file_path.name}")
    print(f"📊 建立時間: {session.creation_time.strftime('%Y-%m-%d %H:%M:%S')}")
else:
    print("❌ 無法初始化會話，請檢查測試檔案")

🚀 初始化實驗會話...
✅ 會話建立成功！
📊 實驗名稱: Unknown
📊 TXT 檔案: 20250521_Janus Stacking SiO2_13K_113.txt
📊 建立時間: 2025-06-07 16:33:10


## 4️⃣ 檢視會話摘要 / View Session Summary

In [5]:
# 顯示會話摘要 - 了解有哪些檔案可用
if 'session' in locals():
    summary = session.get_session_summary()
    files_summary = summary['files_summary']
    
    print("📊 會話摘要 / Session Summary")
    print("=" * 40)
    print(f"實驗名稱: {summary['experiment_name']}")
    print(f"總檔案數: {files_summary['total_available']}")
    print(f"  - TXT: {files_summary['available']['txt']}")
    print(f"  - TOPO: {files_summary['available']['topo']}")
    print(f"  - CITS: {files_summary['available']['cits']}")
    print(f"  - STS: {files_summary['available']['sts']}")
    
    # 顯示掃描參數
    scan_params = summary.get('scan_parameters')
    if scan_params:
        print(f"\n🔬 掃描參數:")
        print(f"  - 像素數: {scan_params['x_pixel']} × {scan_params['y_pixel']}")
        print(f"  - 掃描範圍: {scan_params['x_range']:.1f} × {scan_params['y_range']:.1f} nm")
        print(f"  - X 像素尺度: {scan_params['pixel_scale_x']:.4f} nm/pixel")
        print(f"  - Y 像素尺度: {scan_params['pixel_scale_y']:.4f} nm/pixel")

📊 會話摘要 / Session Summary
實驗名稱: Unknown
總檔案數: 17
  - TXT: 1
  - TOPO: 12
  - CITS: 4
  - STS: 0

🔬 掃描參數:
  - 像素數: 500 × 500
  - 掃描範圍: 10.0 × 10.0 nm
  - X 像素尺度: 0.0200 nm/pixel
  - Y 像素尺度: 0.0200 nm/pixel


## 5️⃣ 新的檔案存取方式 / New File Access Methods

### 舊方式 vs 新方式對比

**❌ 舊方式 (複雜且容易出錯):**
```python
data = analyzer.experiment_data['loaded_files']['topofwd']['data']['image_data']
scale = analyzer.experiment_data['txt_data']['scan_parameters']['x_range']
```

**✅ 新方式 (IDE 友好且直覺):**
```python
topo = session['topofwd']        # FileProxy with full type hints
data = topo.data.image           # TopoData.image: np.ndarray
scale = topo.data.x_range        # TopoData.x_range: float
```

In [6]:
# 列出可用檔案
if 'session' in locals():
    available_files = session.available_files
    
    print("📋 可用檔案列表 / Available Files:")
    print("=" * 40)
    
    for file_type, files in available_files.items():
        if files:
            print(f"\n📁 {file_type.upper()} 檔案 ({len(files)} 個):")
            for i, file_key in enumerate(files[:5]):  # 只顯示前5個
                print(f"   {i+1}. {file_key}")
            if len(files) > 5:
                print(f"   ... 還有 {len(files)-5} 個檔案")

📋 可用檔案列表 / Available Files:

📁 TXT 檔案 (1 個):
   1. 20250521_Janus Stacking SiO2_13K_113

📁 TOPO 檔案 (12 個):
   1. 20250521_Janus Stacking SiO2_13K_113TopoFwd
   2. 20250521_Janus Stacking SiO2_13K_113TopoBwd
   3. 20250521_Janus Stacking SiO2_13K_113Lia1XFwd
   4. 20250521_Janus Stacking SiO2_13K_113Lia1XBwd
   5. 20250521_Janus Stacking SiO2_13K_113Lia1YFwd
   ... 還有 7 個檔案

📁 CITS 檔案 (4 個):
   1. 20250521_Janus Stacking SiO2_13K_113It_to_PC_Matrix
   2. 20250521_Janus Stacking SiO2_13K_113Lia1R_Matrix
   3. 20250521_Janus Stacking SiO2_13K_113Lia1Y_Matrix
   4. 20250521_Janus Stacking SiO2_13K_113Lia2R_Matrix


## 6️⃣ 測試拓撲檔案存取 / Test Topography File Access

In [7]:
# 選擇一個拓撲檔案進行測試
if 'session' in locals() and session.available_files['topo']:
    topo_key = session.available_files['topo'][0]
    print(f"🔍 測試檔案: {topo_key}")
    print("=" * 40)
    
    # 使用新的直覺存取方式
    topo = session[topo_key]
    
    print(f"📊 檔案資訊:")
    print(f"  - 檔案類型: {topo.file_type}")
    print(f"  - 已載入: {topo.is_loaded}")
    
    if topo.file_info:
        info = topo.file_info
        print(f"  - 檔案大小: {info.human_readable_size}")
        print(f"  - 訊號類型: {info.signal_type}")
        print(f"  - 掃描方向: {info.direction}")
else:
    print("❌ 沒有可用的拓撲檔案")

🔍 測試檔案: 20250521_Janus Stacking SiO2_13K_113TopoFwd
📊 檔案資訊:
  - 檔案類型: topo
  - 已載入: False
  - 檔案大小: 976.6 KB
  - 訊號類型: Topo
  - 掃描方向: Fwd


In [8]:
# 載入並檢視拓撲數據
if 'topo' in locals():
    try:
        print("📥 載入拓撲數據...")
        data = topo.data  # 這會自動觸發載入
        
        if isinstance(data, TopoData):
            print("\n✅ 數據載入成功！")
            print(f"📊 拓撲數據資訊:")
            print(f"  - 圖像尺寸: {data.shape}")
            print(f"  - X 範圍: {data.x_range:.2f} nm")
            print(f"  - Y 範圍: {data.y_range:.2f} nm")
            print(f"  - X 像素尺度: {data.pixel_scale_x:.4f} nm/pixel")
            print(f"  - Y 像素尺度: {data.pixel_scale_y:.4f} nm/pixel")
            print(f"  - 數值 Scale: {data.data_scale}")
            print(f"  - 訊號類型: {data.signal_type}")
            print(f"  - 掃描方向: {data.direction}")
            
            # 顯示數據統計
            image = data.image
            print(f"\n📈 數據統計:")
            print(f"  - 最小值: {image.min():.6f}")
            print(f"  - 最大值: {image.max():.6f}")
            print(f"  - 平均值: {image.mean():.6f}")
            print(f"  - 標準差: {image.std():.6f}")
            
    except Exception as e:
        print(f"❌ 載入失敗: {str(e)}")
        import traceback
        traceback.print_exc()

📥 載入拓撲數據...

✅ 數據載入成功！
📊 拓撲數據資訊:
  - 圖像尺寸: (500, 500)
  - X 範圍: 10.00 nm
  - Y 範圍: 10.00 nm
  - X 像素尺度: 0.0200 nm/pixel
  - Y 像素尺度: 0.0200 nm/pixel
  - 數值 Scale: -2.60913687478663e-07
  - 訊號類型: Topo
  - 掃描方向: Fwd

📈 數據統計:
  - 最小值: -93.305322
  - 最大值: -91.885328
  - 平均值: -92.788010
  - 標準差: 0.197238


## 7️⃣ 測試 CITS 檔案存取 / Test CITS File Access

In [9]:
# 測試 CITS 檔案（如果有的話）
if 'session' in locals() and session.available_files['cits']:
    cits_key = session.available_files['cits'][0]
    print(f"🔬 測試 CITS 檔案: {cits_key}")
    print("=" * 40)
    
    try:
        cits = session[cits_key]
        print(f"📊 CITS 檔案資訊:")
        print(f"  - 檔案類型: {cits.file_type}")
        print(f"  - 已載入: {cits.is_loaded}")
        
        # 載入 CITS 數據
        print("\n📥 載入 CITS 數據...")
        cits_data = cits.data
        
        if isinstance(cits_data, CitsData):
            print("✅ CITS 數據載入成功！")
            print(f"  - 3D 數據形狀: {cits_data.shape}")
            print(f"  - 偏壓點數: {cits_data.n_bias_points}")
            print(f"  - 偏壓範圍: {cits_data.bias_range[0]:.3f} ~ {cits_data.bias_range[1]:.3f} V")
            print(f"  - 網格大小: {cits_data.grid_size}")
            print(f"  - X 範圍: {cits_data.x_range:.2f} nm")
            print(f"  - Y 範圍: {cits_data.y_range:.2f} nm")
        
    except Exception as e:
        print(f"❌ CITS 載入失敗: {str(e)}")
else:
    print("ℹ️  沒有可用的 CITS 檔案")

🔬 測試 CITS 檔案: 20250521_Janus Stacking SiO2_13K_113It_to_PC_Matrix
📊 CITS 檔案資訊:
  - 檔案類型: cits
  - 已載入: False

📥 載入 CITS 數據...
✅ CITS 數據載入成功！
  - 3D 數據形狀: (401, 100, 100)
  - 偏壓點數: 401
  - 偏壓範圍: -2050.000 ~ 1050.000 V
  - 網格大小: [100, 100]
  - X 範圍: 100.00 nm
  - Y 範圍: 100.00 nm


/Users/yangziliang/Git-Projects/keen/backend/core/parsers/dat_parser.py:84: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t', header=None)


## 8️⃣ 分析器功能測試 / Analyzer Features Test

In [10]:
# 測試分析器功能
if 'topo' in locals():
    print("🔧 測試分析器功能")
    print("=" * 40)
    
    try:
        # 獲取分析器
        analyzer = topo.analyzer
        print(f"✅ 分析器類型: {type(analyzer).__name__}")
        
        # 顯示可用的分析方法
        print("\n🔧 可用的分析方法:")
        print("  - topo.flatten_plane()      # 平面平坦化")
        print("  - topo.extract_profile()    # 提取剖面線")
        print("  - topo.analyzer             # 獲取完整分析器")
        
        # 檢查分析器是否有常用方法
        methods = [attr for attr in dir(analyzer) if not attr.startswith('_')]
        print(f"\n📋 分析器方法 ({len(methods)} 個):")
        for method in methods[:10]:  # 只顯示前10個
            print(f"  - {method}")
        if len(methods) > 10:
            print(f"  ... 還有 {len(methods)-10} 個方法")
        
    except Exception as e:
        print(f"❌ 分析器測試失敗: {str(e)}")
else:
    print("❌ 沒有可用的拓撲檔案進行分析器測試")

🔧 測試分析器功能
✅ 分析器類型: IntAnalyzer

🔧 可用的分析方法:
  - topo.flatten_plane()      # 平面平坦化
  - topo.extract_profile()    # 提取剖面線
  - topo.analyzer             # 獲取完整分析器

📋 分析器方法 (25 個):
  - analysis_history
  - analyze
  - apply_flattening
  - apply_tilt_correction
  - cached_results
  - clear_cache
  - current_line_profile
  - current_topo_data
  - detect_features
  - extract_line_profile
  ... 還有 15 個方法


## 9️⃣ 批次操作和搜尋功能 / Batch Operations and Search Features

In [11]:
# 測試批次操作和搜尋功能
if 'session' in locals():
    print("📁 批次操作和搜尋功能測試")
    print("=" * 40)
    
    # 獲取所有拓撲檔案
    topo_files = session.get_topo_files()
    print(f"📂 拓撲檔案 ({len(topo_files)} 個):")
    for i, file_key in enumerate(topo_files[:5]):
        file_proxy = session[file_key]
        info = file_proxy.file_info
        signal_type = info.signal_type if info else "Unknown"
        direction = info.direction if info else ""
        size = info.human_readable_size if info else "Unknown"
        print(f"  {i+1}. {file_key} ({signal_type} {direction}) - {size}")
    
    if len(topo_files) > 5:
        print(f"  ... 還有 {len(topo_files)-5} 個檔案")
    
    # 根據訊號類型搜尋
    print("\n🔍 搜尋功能測試:")
    topo_signal_files = session.find_files_by_signal_type("Topo")
    print(f"  - Topo 訊號檔案: {len(topo_signal_files)} 個")
    
    # 根據方向搜尋
    fwd_files = session.find_files_by_direction("Fwd")
    bwd_files = session.find_files_by_direction("Bwd")
    print(f"  - 正向掃描檔案: {len(fwd_files)} 個")
    print(f"  - 反向掃描檔案: {len(bwd_files)} 個")

📁 批次操作和搜尋功能測試
📂 拓撲檔案 (12 個):
  1. 20250521_Janus Stacking SiO2_13K_113TopoFwd (Topo Fwd) - 976.6 KB
  2. 20250521_Janus Stacking SiO2_13K_113TopoBwd (Topo Bwd) - 976.6 KB
  3. 20250521_Janus Stacking SiO2_13K_113Lia1XFwd (Lia1X Fwd) - 976.6 KB
  4. 20250521_Janus Stacking SiO2_13K_113Lia1XBwd (Lia1X Bwd) - 976.6 KB
  5. 20250521_Janus Stacking SiO2_13K_113Lia1YFwd (Lia1Y Fwd) - 976.6 KB
  ... 還有 7 個檔案

🔍 搜尋功能測試:
  - Topo 訊號檔案: 2 個
  - 正向掃描檔案: 6 個
  - 反向掃描檔案: 6 個


## 🔟 記憶體使用資訊 / Memory Usage Information

In [12]:
# 檢視記憶體使用資訊
if 'session' in locals():
    print("💾 記憶體使用資訊")
    print("=" * 40)
    
    memory_info = session.get_memory_info()
    
    print(f"📊 整體狀況:")
    print(f"  - 總檔案數: {memory_info['total_files']}")
    print(f"  - 已載入檔案: {memory_info['total_loaded']}")
    print(f"  - 檔案代理快取: {memory_info['proxy_cache_size']} 個")
    
    print(f"\n📈 各管理器快取狀況:")
    for manager_name, cache_info in memory_info.items():
        if isinstance(cache_info, dict) and 'cache_size' in cache_info:
            hit_rate = cache_info.get('hit_rate', 0) * 100
            print(f"  - {manager_name}: {cache_info['cache_size']} 個 (命中率: {hit_rate:.1f}%)")
    
    # 顯示已載入檔案
    loaded_files = session.loaded_files
    total_loaded = sum(len(files) for files in loaded_files.values())
    if total_loaded > 0:
        print(f"\n📂 已載入檔案詳情:")
        for file_type, files in loaded_files.items():
            if files:
                print(f"  - {file_type.upper()}: {files}")

💾 記憶體使用資訊
📊 整體狀況:
  - 總檔案數: 17
  - 已載入檔案: 3
  - 檔案代理快取: 6 個

📈 各管理器快取狀況:
  - txt_cache: 1 個 (命中率: 80.0%)
  - topo_cache: 1 個 (命中率: 0.0%)
  - cits_cache: 1 個 (命中率: 0.0%)
  - sts_cache: 0 個 (命中率: 0.0%)

📂 已載入檔案詳情:
  - TXT: ['20250521_Janus Stacking SiO2_13K_113']
  - TOPO: ['20250521_Janus Stacking SiO2_13K_113TopoFwd']
  - CITS: ['20250521_Janus Stacking SiO2_13K_113It_to_PC_Matrix']


## 1️⃣1️⃣ 新舊架構完整對比 / Complete Old vs New Architecture Comparison

### 🔄 程式碼對比 / Code Comparison

#### 📋 **舊架構存取方式 / Old Architecture Access:**

```python
# ❌ 複雜的字典式存取，無 IDE 支援
analyzer = MainAnalyzer()
result = analyzer.load_experiment('experiment.txt')
data = analyzer.experiment_data['loaded_files']['topofwd']['data']['image_data']
scale = analyzer.experiment_data['txt_data']['scan_parameters']['x_range']

# 需要手動管理狀態
# 容易拼寫錯誤
# 沒有型別提示
# 狀態可能遺失
```

#### ✨ **新架構存取方式 / New Architecture Access:**

```python
# ✅ 直覺的屬性存取，完整 IDE 支援
session = ExperimentSession('experiment.txt')
topo = session['topofwd']           # FileProxy with full type hints
data = topo.data.image              # TopoData.image: np.ndarray
scale = topo.data.x_range           # TopoData.x_range: float

# 自動狀態管理
# 型別安全
# IDE 自動完成
# 統一錯誤處理
# 正確的 scale 處理
```

## 1️⃣2️⃣ 總結與下一步 / Summary and Next Steps

In [13]:
# 最終總結
print("✨ 新架構優勢總結 / New Architecture Advantages")
print("=" * 60)
print("✅ IDE 友好的型別提示 - 完整的自動完成和錯誤檢查")
print("✅ 直覺的屬性存取 - session['file'].data.attribute")
print("✅ 完整的狀態管理 - 所有分析結果自動持久化")
print("✅ 統一的資料格式 - 標準化的 ParseResult 和錯誤處理")
print("✅ 清晰的職責分離 - 型別管理器模式")
print("✅ 正確的 Scale 處理 - 從 TXT 檔案獲取正確的數值 scale")
print("✅ 智能快取管理 - 自動最佳化記憶體使用")
print("✅ 批次操作支援 - 高效的多檔案處理")
print("✅ 強大的搜尋功能 - 根據訊號類型和方向快速定位檔案")

print("\n🚀 建議的下一步:")
print("1. 根據您的具體需求測試各種檔案類型")
print("2. 嘗試使用分析器進行實際的數據處理")
print("3. 探索批次處理功能來處理多個檔案")
print("4. 使用搜尋功能來快速定位特定類型的檔案")
print("5. 如有問題，參考 docs/migration_guide_v2.md")

✨ 新架構優勢總結 / New Architecture Advantages
✅ IDE 友好的型別提示 - 完整的自動完成和錯誤檢查
✅ 直覺的屬性存取 - session['file'].data.attribute
✅ 完整的狀態管理 - 所有分析結果自動持久化
✅ 統一的資料格式 - 標準化的 ParseResult 和錯誤處理
✅ 清晰的職責分離 - 型別管理器模式
✅ 正確的 Scale 處理 - 從 TXT 檔案獲取正確的數值 scale
✅ 智能快取管理 - 自動最佳化記憶體使用
✅ 批次操作支援 - 高效的多檔案處理
✅ 強大的搜尋功能 - 根據訊號類型和方向快速定位檔案

🚀 建議的下一步:
1. 根據您的具體需求測試各種檔案類型
2. 嘗試使用分析器進行實際的數據處理
3. 探索批次處理功能來處理多個檔案
4. 使用搜尋功能來快速定位特定類型的檔案
5. 如有問題，參考 docs/migration_guide_v2.md


---

## 📝 實驗區域 / Experimental Section

您可以在下面的 cell 中自由測試新架構的各種功能：

In [ ]:
# 🧪 自由實驗區域 - 在這裡測試任何您想要的功能
# 🧪 Free experimental area - test any features you want here

if 'session' in locals():
    print("🧪 實驗環境已準備就緒！")
    print(f"📋 可用檔案: {sum(len(files) for files in session.available_files.values())} 個")
    
    # 例子 1: 測試檔案快速存取
    if session.available_files['topo']:
        first_topo = session[session.available_files['topo'][0]]
        print(f"📊 快速存取測試: {first_topo}")
    
    # 例子 2: 測試記憶體管理
    print(f"💾 目前記憶體狀態: {session.get_memory_info()['total_loaded']} 個檔案已載入")
    
    # 例子 3: 測試 Scale 修正 - 顯示正確的數值 scale
    if 'topo' in locals():
        print(f"🔬 Scale 資訊測試:")
        print(f"  - 數值 Scale (從 TXT): {topo.data.data_scale}")
        print(f"  - X 像素尺度: {topo.data.pixel_scale_x:.4f} nm/pixel")
        print(f"  - Y 像素尺度: {topo.data.pixel_scale_y:.4f} nm/pixel")
    
    # 在這裡添加您的實驗程式碼
    # Add your experimental code here
    
else:
    print("❌ 請先執行上面的初始化程式碼")